In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
import os
import json

# Import API keys
from config import blsapi

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_directory = "output_data"
output_data_file = "authjobs.csv"
output_full_path = output_directory + "/" + output_data_file

# Define the base URL
# url = "https://authenticjobs.com/api/?"

# Build partial query URL
# query_url = f"{url}api_key={authapi}"


# joblist_url = f"{query_url}&method=aj.types.getlist"
# catelist_url = f"{query_url}&method=aj.categories.getlist&format=json"

# print(catelist_url)

# Format for series id parameter - this site defines the series id format that must be used
# https://www.bls.gov/help/hlpforma.htm#OE
# this analysis focuses on Occupational Employment statistics

# 	Series ID    OEUN000000011100011000001
# 	Positions       Value           Field Name
# 	1-2             OE              Prefix
# 	3               U               Seasonal Adjustment Code
prefix = 'OE'
season_ad_cd = 'U'



# Area codes are defined
# https://download.bls.gov/pub/time.series/oe/oe.area
# 	4               N               Area Type Code
# 	5-11            0000000         Area Code
area_type_code = 'N'
area_code = '0000000'

# Industry codes are defined
# https://download.bls.gov/pub/time.series/oe/oe.industry
# 	12-17           111000          Industry Code
ind_code = '000000'

# We have decided to focus on the following occupations
# OCC_CODE	OCC_TITLE
# 13-1141	Compensation, Benefits, and Job Analysis Specialists
# 13-1161	Market Research Analysts and Marketing Specialists
# 13-2031	Budget Analysts
# 13-2041	Credit Analysts
# 13-2051	Financial Analysts
# 15-1121	Computer Systems Analysts
# 15-1122	Information Security Analysts
# 15-1141	Database Administrators
# 15-2011	Actuaries
# 15-2031	Operations Research Analysts
# 15-2041	Statisticians
# 	18-23           110000          Occupation Code
occ_code = ['131141','131161','132031','132041','132051','151121','151122','151141','152011','152031','152041']
occ_title = ['Compensation, Benefits, and Job Analysis Specialists','Market Research Analysts and Marketing Specialists',
            'Budget Analysts','Credit Analysts','Financial Analysts','Computer Systems Analysts','Information Security Analysts',
            'Database Administrators','Actuaries', 'Operations Research Analysts', 'Statisticians']

data_set = pd.DataFrame({"Occupation Code": occ_code,
           "Occupation Title": occ_title})

# The list of data types can be found
# https://download.bls.gov/pub/time.series/oe/oe.datatype
# 	24-25           01              Data Type Code
# 01	Employment
# 04	Annual mean wage	
# 13	Annual median wage	
# 16	Employment per 1,000 jobs	
# 17	Location Quotient	
# based on the data limitations the API call will be one for each data type.  Then 
data_types = ['01']
data_type_name = ['Employment']


# build series list
# per API restrictions a single API call cannot contain more than 50 series IDs
series_list = []
for occup in occ_code:
    for infor in data_types:
        series_list.append(prefix + season_ad_cd + area_type_code + area_code + ind_code + occup + infor)


headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": series_list,"startyear":"2017", "endyear":"2017", "registrationkey": blsapi})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)



In [112]:
seriesid = []
keyvalue = []

data_type_name = ['Employment']

print(len(data_type_name))
for x in range(len(json_data['Results']['series'])):
    seriesid.append(json_data['Results']['series'][x]['seriesID'][-8:-2])
    keyvalue.append(json_data['Results']['series'][x]['data'][0]['value'])
    
data_collector = pd.DataFrame({"Occupation Code": seriesid,
           data_type_name[0]: keyvalue})
    
merge_table = pd.merge(data_set, data_collector, on="Occupation Code")

data_collector = pd.DataFrame([], data_collector.index, data_collector.columns)
data_collector = data_collector.dropna()
print(data_collector)
merge_table
# json_data_set = pd.DataFrame({"Occupation Code": seriesid,
#                               "Employment": keyvalue})
# json_data_set

Empty DataFrame
Columns: [Occupation Code, Employment]
Index: []


,Occupation Code,Occupation Title,Employment
0,131141,"Compensation, Benefits, and Job Analysis Speci...",80530
1,131161,Market Research Analysts and Marketing Special...,596450
2,132031,Budget Analysts,54550
3,132041,Credit Analysts,74850
4,132051,Financial Analysts,294110
5,151121,Computer Systems Analysts,581960
6,151122,Information Security Analysts,105250
7,151141,Database Administrators,113690
8,152011,Actuaries,19210
9,152031,Operations Research Analysts,106050


In [31]:
# result_set.head()
# json_data
# test = json_data['Results']['series'][1]['seriesID'][-8:-2]
# print(test)
# # print(test[-8:-2])
# json_data['Results']['series'][1]['data'][0]['value']


requests.models.Response

In [33]:
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)